## Goal

- train the example RBP model
  - nothing fancy
  - no concise dependencies 
  - just single-task for simplicity - PUM2
- export the model to .json and weights to hdf5  


## Structure

Input files:

- fasta
- Bed
- annotation GTF

Input features:

- 1-hot-encoded sequence
- annotation features & distances

I will start with the batch preprocessor (loading the whole dataset at once) and then add a generator.

## Open questions

- difference between genomelake and genomedatalayer?
- I really like genomelake - it should become the standard for the pre-processors
  - should be simple enough to use and understand
  - add nearest feature-point extractor
- test_files should be raw files not already pre-processed folders

## Pre-processor steps

- Given range extract the sequence from the fasta file
  - Validate the width (or just extract the center)
  - compute the center range for it
  - pybed tools
    - [...] check the kundaje-lab code for this task

- Load-in the GTF file and compute the distances to the nearest features
  - [ ] see the programming possibilities for doing this in python
  - see 4_append_other_positions.R
  - just use pandas.DataFrame for it
    - Idea: include it into concise as a pre-processor
    
- Why are they not using HTSeq?    
  

In [1]:
import pandas as pd
import pickle
from concise.preprocessing import encodeDNA, EncodeSplines

Using TensorFlow backend.


In [2]:
import concise.layers as cl
import keras.layers as kl
import concise.initializers as ci
import concise.regularizers as cr
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import EarlyStopping

In [4]:
CONCISE_ROOT = "/home/avsec/projects-work/concise/"
def load(split="train", st=None):
    dt = pd.read_csv(CONCISE_ROOT + "/data/RBP/PUM2_{0}.csv".format(split))
    # DNA/RNA sequence
    xseq = encodeDNA(dt.seq) 
    # distance to the poly-A site
    xpolya = dt.polya_distance.as_matrix().reshape((-1, 1))
    # response variable
    y = dt.binding_site.as_matrix().reshape((-1, 1)).astype("float")
    return {"seq": xseq, "dist_polya_raw": xpolya}, y

def data():
    
    train, valid, test = load("train"), load("valid"), load("test")
    
    # transform the poly-A distance with B-splines
    es = EncodeSplines()
    es.fit(train[0]["dist_polya_raw"])
    train[0]["dist_polya_st"] = es.transform(train[0]["dist_polya_raw"], warn=False)
    valid[0]["dist_polya_st"] = es.transform(valid[0]["dist_polya_raw"], warn=False)
    test[0]["dist_polya_st"] = es.transform(test[0]["dist_polya_raw"], warn=False)
    
    #return load("train"), load("valid"), load("test")
    return train + (es,), valid, test

train, valid, test = data()

In [11]:
# TODO - pass with the model (encodeSplines), trained pre-processor
train[2]

In [50]:
def model(train, filters=1, kernel_size=9, pwm_list=None, lr=0.001, ext_dist=False):
    seq_length = train[0]["seq"].shape[1]
    if pwm_list is None:
        kinit = "glorot_uniform"
        binit = "zeros"
    else:
        kinit = ci.PSSMKernelInitializer(pwm_list, add_noise_before_Pwm2Pssm=True)
        binit = "zeros"
        
    # sequence
    in_dna = cl.InputDNA(seq_length=seq_length, name="seq")
    inputs = [in_dna]
    x = kl.Conv1D(filters=filters, 
                  kernel_size=kernel_size, 
                  activation="relu",
                  kernel_initializer=kinit,
                  bias_initializer=binit,
                  name="conv1")(in_dna)
    x = kl.AveragePooling1D(pool_size=4)(x)
    x = kl.Flatten()(x)
        
    if ext_dist:    
        # distance
        in_dist = kl.Input(train[0]["dist_polya_st"].shape[1:], name="dist_polya_st")
        x_dist = cl.SplineT()(in_dist)
        x = kl.concatenate([x, x_dist])
        inputs += [in_dist]
    
    x = kl.Dense(units=1)(x)
    m = Model(inputs, x)
    m.compile(Adam(lr=lr), loss="binary_crossentropy", metrics=["acc"])
    return m

In [58]:
m = model(train, filters=10, ext_dist=True)

In [60]:
m.fit(train[0], train[1], epochs=50, validation_data=valid, 
      callbacks=[EarlyStopping(patience=5)])

Train on 17713 samples, validate on 4881 samples
Epoch 1/50
17713/17713 [==============================] - 2s - loss: 0.3946 - acc: 0.8498 - val_loss: 0.4038 - val_acc: 0.8617
Epoch 2/50
17713/17713 [==============================] - 2s - loss: 0.3544 - acc: 0.8661 - val_loss: 0.4033 - val_acc: 0.8566
Epoch 3/50
17713/17713 [==============================] - 2s - loss: 0.3441 - acc: 0.8665 - val_loss: 0.3994 - val_acc: 0.8513
Epoch 4/50
17713/17713 [==============================] - 2s - loss: 0.3335 - acc: 0.8694 - val_loss: 0.4039 - val_acc: 0.8486
Epoch 5/50
17713/17713 [==============================] - 2s - loss: 0.3317 - acc: 0.8648 - val_loss: 0.4137 - val_acc: 0.8486
Epoch 6/50
17713/17713 [==============================] - 2s - loss: 0.3301 - acc: 0.8628 - val_loss: 0.3912 - val_acc: 0.8513
Epoch 7/50
17713/17713 [==============================] - 2s - loss: 0.3587 - acc: 0.8587 - val_loss: 0.4358 - val_acc: 0.8574
Epoch 8/50
17713/17713 [==============================] - 2s -

In [61]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
seq (InputLayer)                 (None, 101, 4)        0                                            
____________________________________________________________________________________________________
conv1 (Conv1D)                   (None, 93, 10)        370         seq[0][0]                        
____________________________________________________________________________________________________
average_pooling1d_6 (AveragePool (None, 23, 10)        0           conv1[0][0]                      
____________________________________________________________________________________________________
dist_polya_st (InputLayer)       (None, 1, 10)         0                                            
___________________________________________________________________________________________

In [62]:
m.save_weights("model/weights.h5")

In [63]:
model_json = m.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)

In [64]:
train[2]

## Test loading

In [5]:
with open("preprocessor/encodeSplines.pkl", "wb") as output_file:
    pickle.dump(train[2], output_file)

In [66]:
es_file = "preprocessor/encodeSplines.pkl"

In [67]:
with open(es_file, "rb") as f:
    es2 = pickle.load(f)

In [225]:
def read_model(arch="model/model.json", weights="model/weights.h5"):
    from keras.models import model_from_json
    with open(arch, 'r') as f:
        m = model_from_json(f.read())
    m.load_weights("model/weights.h5")
    return m

In [44]:
m = read_model()

In [45]:
m.predict(train[0])

array([[-0.2006],
       [ 0.4689],
       [ 0.4009],
       ..., 
       [-0.0013],
       [ 0.1539],
       [-0.0068]], dtype=float32)

## Test the whole pipeline

In [43]:
def read_model(arch="model/model.json", weights="model/weights.h5"):
    from keras.models import model_from_json
    with open(arch, 'r') as f:
        m = model_from_json(f.read())
    m.load_weights("model/weights.h5")
    return m

In [226]:
# pre-processor
from concise.utils.helper import read_json
from preprocessor import preprocessor
# kwargs
pp_kwargs = read_json("preprocessor_test_kwargs.json")
pp = preprocessor(**pp_kwargs)

In [221]:
# model
m = read_model()

In [222]:
# unpack
pp_inputs = map(lambda x: x["inputs"], pp)

In [223]:
res = np.concatenate([m.predict_on_batch(next(pp_inputs)) for i in range(3)])

INFO:2017-08-11 12:01:02,771:genomelake] Running landmark extractors..
2017-08-11 12:01:02,771 [INFO] Running landmark extractors..
INFO:2017-08-11 12:01:02,785:genomelake] Done!
2017-08-11 12:01:02,785 [INFO] Done!


In [224]:
res

array([[ 0.2593],
       [ 0.2135],
       [ 0.2135],
       [ 0.2593],
       [ 0.2593],
       [ 0.2593],
       [ 0.2135],
       [ 0.2593],
       [ 0.2593],
       [ 0.2593],
       [ 0.2593],
       [ 0.2593]], dtype=float32)

In [208]:
# TODO - troubled
#res = m.predict_generator(pp_inputs, steps=3)

## TODO 

- pack everything into the pipeline